In [ ]:
!wget https://www.dropbox.com/s/9sakh5x8h1rrrip/HW4.zip

In [ ]:
!mkdir data
!unzip HW4.zip -d ./data

# Imports

In [ ]:
import cv2
import numpy as np
from glob import glob
from scipy import ndimage
import matplotlib.pyplot as plt

# Utilities

In [ ]:
# Compute the convolution of image with a filter
def conv(image, filter, pad=0):
    if pad > 0:
        nimage = np.zeros((image.shape[0]+2*pad, image.shape[1]+2*pad))
        nimage[pad:-pad, pad:-pad] = image
    else:
        nimage = image.copy()
    h, w = nimage.shape
    kh, kw = filter.shape
    resp = np.zeros((h-kh+1, w-kw+1))
    for i in range(h-kh+1):
        for j in range(w-kw+1):
            resp[i, j] = np.sum(nimage[i:i+kh, j:j+kw] * filter)
    return resp

# Create a gaussian filter
def gaussian_kernel(size, sigma):
    assert size % 2 == 1
    limit = int(size / 2)
    x = np.arange(-limit, limit+1).reshape((1, -1))
    y = np.arange(-limit, limit+1).reshape((-1, 1))
    kernel = np.exp(- (x ** 2 + y ** 2) / (2 * sigma))
    kernel /= kernel.sum()
    return kernel

# Apply horizontal and vertical sobel operators
def SobelFilter(img, direction):
    if (direction == 'x'):
        Gx = np.array([[-1,0,+1], [-2,0,+2],  [-1,0,+1]])
        Res = ndimage.convolve(img, Gx)
    
    if (direction == 'y'):
        Gy = np.array([[-1,-2,-1], [0,0,0], [+1,+2,+1]])
        Res = ndimage.convolve(img, Gy)
    
    return Res

# Normalize an image
def normalize(img):
    img = img/img.max()
    return img

In [ ]:

# Do non maximum suppression
def nms(Gmag, Grad, Gx, Gy):
    NMS = np.zeros(Gmag.shape)
    for i in range(1, int(Gmag.shape[0]) - 1):
        for j in range(1, int(Gmag.shape[1]) - 1):
            if((Grad[i,j] >= 0 and Grad[i,j] <= 45) or (Grad[i,j] < -135 and Grad[i,j] >= -180)):
                yBot = np.array([Gmag[i,j+1], Gmag[i+1,j+1]])
                yTop = np.array([Gmag[i,j-1], Gmag[i-1,j-1]])
                x_est = np.absolute(Gy[i,j]/Gmag[i,j])
                if (Gmag[i,j] >= ((yBot[1]-yBot[0])*x_est+yBot[0]) and Gmag[i,j] >= ((yTop[1]-yTop[0])*x_est+yTop[0])):
                    NMS[i,j] = Gmag[i,j]
                else:
                    NMS[i,j] = 0
            if((Grad[i,j] > 45 and Grad[i,j] <= 90) or (Grad[i,j] < -90 and Grad[i,j] >= -135)):
                yBot = np.array([Gmag[i+1,j] ,Gmag[i+1,j+1]])
                yTop = np.array([Gmag[i-1,j] ,Gmag[i-1,j-1]])
                x_est = np.absolute(Gx[i,j]/Gmag[i,j])
                if (Gmag[i,j] >= ((yBot[1]-yBot[0])*x_est+yBot[0]) and Gmag[i,j] >= ((yTop[1]-yTop[0])*x_est+yTop[0])):
                    NMS[i,j] = Gmag[i,j]
                else:
                    NMS[i,j] = 0
            if((Grad[i,j] > 90 and Grad[i,j] <= 135) or (Grad[i,j] < -45 and Grad[i,j] >= -90)):
                yBot = np.array([Gmag[i+1,j] ,Gmag[i+1,j-1]])
                yTop = np.array([Gmag[i-1,j] ,Gmag[i-1,j+1]])
                x_est = np.absolute(Gx[i,j]/Gmag[i,j])
                if (Gmag[i,j] >= ((yBot[1]-yBot[0])*x_est+yBot[0]) and Gmag[i,j] >= ((yTop[1]-yTop[0])*x_est+yTop[0])):
                    NMS[i,j] = Gmag[i,j]
                else:
                    NMS[i,j] = 0
            if((Grad[i,j] > 135 and Grad[i,j] <= 180) or (Grad[i,j] < 0 and Grad[i,j] >= -45)):
                yBot = np.array([Gmag[i,j-1] ,Gmag[i+1,j-1]])
                yTop = np.array([Gmag[i,j+1] ,Gmag[i-1,j+1]])
                x_est = np.absolute(Gy[i,j]/Gmag[i,j])
                if (Gmag[i,j] >= ((yBot[1]-yBot[0])*x_est+yBot[0]) and Gmag[i,j] >= ((yTop[1]-yTop[0])*x_est+yTop[0])):
                    NMS[i,j] = Gmag[i,j]
                else:
                    NMS[i,j] = 0
    return NMS

In [ ]:
# Apply double thresholding
def double_th(img, weak_th, strong_th):

    highThresholdRatio = strong_th
    lowThresholdRatio = weak_th 
    GSup = np.copy(img)
    h = int(GSup.shape[0])
    w = int(GSup.shape[1])
    highThreshold = np.max(GSup) * highThresholdRatio
    lowThreshold = highThreshold * lowThresholdRatio    
    x = 0.1
    oldx=0

    while(oldx != x):
        oldx = x
        for i in range(1,h-1):
            for j in range(1,w-1):
                if(GSup[i,j] > highThreshold):
                    GSup[i,j] = 1
                elif(GSup[i,j] < lowThreshold):
                    GSup[i,j] = 0
                else:
                    if((GSup[i-1,j-1] > highThreshold) or 
                        (GSup[i-1,j] > highThreshold) or
                        (GSup[i-1,j+1] > highThreshold) or
                        (GSup[i,j-1] > highThreshold) or
                        (GSup[i,j+1] > highThreshold) or
                        (GSup[i+1,j-1] > highThreshold) or
                        (GSup[i+1,j] > highThreshold) or
                        (GSup[i+1,j+1] > highThreshold)):
                        GSup[i,j] = 1
        x = np.sum(GSup == 1)
    
    GSup = (GSup == 1) * GSup
    return GSup

# Sobel

In [ ]:
sobel_hor = np.array([[-1, 0, 1], 
                      [-2, 0, 2], 
                      [-1, 0, 1]])

sobel_ver = np.array([[-1, -2, -1], 
                      [0, 0, 0], 
                      [1, 2, 1]])

In [ ]:
image_paths = glob('./data/edge folder/*')

for ip in image_paths[:]:
    img = cv2.imread(ip, cv2.IMREAD_GRAYSCALE).astype(np.float32)
    edges_x = conv(img, sobel_hor, pad=1)
    edges_y = conv(img, sobel_ver, pad=1)
    edges = (edges_x ** 2 + edges_y ** 2) ** 0.5
    edges = (edges - edges.min()) / (edges.max() - edges.min())

    fig = plt.figure(figsize=(16, 8))
    ax0 = plt.subplot(1, 2, 1)
    ax0.axis('off')
    ax0.imshow(img, cmap='gray')
    ax1 = plt.subplot(1, 2, 2)
    ax1.axis('off')

    ax1.imshow(edges, cmap='gray')
    plt.show()

# Prewitt

In [ ]:
prewitt_hor = np.array([[-1, 0, 1], 
                        [-1, 0, 1], 
                        [-1, 0, 1]])

prewitt_ver = np.array([[-1, -1, -1], 
                        [0, 0, 0], 
                        [1, 1, 1]])

In [ ]:
image_paths = glob('./data/edge folder/*')
for ip in image_paths[:]:
    img = cv2.imread(ip, cv2.IMREAD_GRAYSCALE).astype(np.float32)
    edges_x = conv(img, prewitt_hor)
    edges_y = conv(img, prewitt_ver)
    edges = (edges_x ** 2 + edges_y ** 2) ** 0.5
    edges = (edges - edges.min()) / (edges.max() - edges.min())

    fig = plt.figure(figsize=(16, 8))
    ax0 = plt.subplot(1, 2, 1)
    ax0.axis('off')
    ax0.imshow(img, cmap='gray')
    ax1 = plt.subplot(1, 2, 2)
    ax1.axis('off')

    ax1.imshow(edges, cmap='gray')
    plt.show()

# Canny

In [ ]:

# Apply canny edge detection
def canny(img, weak_th=None, strong_th=None):
    
    gk = gaussian_kernel(5, 1.4)
    img = conv(img, gk, pad=2)
    
    gx = SobelFilter(img, 'x')
    gx = normalize(gx)
    gy = SobelFilter(img, 'y')
    gy = normalize(gy)

    mag = np.hypot(gx,gy)
    mag = normalize(mag)
    grad = np.degrees(np.arctan2(gy,gx))

    nms_img = nms(mag, grad, gx, gy)
    nms_img = normalize(nms_img)

    result = double_th(nms_img, 0.1, 0.4)
    return result

In [ ]:
image_paths = glob('./data/edge folder/*')
for ip in image_paths[:]:
    img = cv2.imread(ip, cv2.IMREAD_GRAYSCALE).astype(np.float32)

    img = normalize(img)
    edges = canny(img)

    fig = plt.figure(figsize=(16, 8))
    ax0 = plt.subplot(1, 2, 1)
    ax0.axis('off')
    ax0.imshow(img, cmap='gray')
    ax1 = plt.subplot(1, 2, 2)
    ax1.axis('off')

    ax1.imshow(edges, cmap='gray')
    plt.show()

# LoG

In [ ]:
# log = np.array([[-1, -1, -1], 
#                 [-1, 8, -1], 
#                 [-1, -1, -1]])

log = np.array([[0, 0, -1, 0, 0],
                [0, -1, -2, -1, 0],
                [-1, -2, 16, -2, -1],
                [0, -1, -2, -1, 0],
                [0, 0, -1, 0, 0]
                ])

In [ ]:
image_paths = glob('./data/edge folder/*')
for ip in image_paths[:]:

    img = cv2.imread(ip, cv2.IMREAD_GRAYSCALE).astype(np.float32)
    img = normalize(img)
    edges = conv(img, log, pad=2)
    edges = normalize(edges)

    fig = plt.figure(figsize=(16, 8))
    ax0 = plt.subplot(1, 2, 1)
    ax0.axis('off')
    ax0.imshow(img, cmap='gray')
    ax1 = plt.subplot(1, 2, 2)
    ax1.axis('off')

    ax1.imshow(edges, cmap='gray')
    plt.show()